In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver

url_main = 'https://comic.naver.com/' # webtoon/finish 완결, webtoon/detail?titleId=xxx&no=1 1화
driver_path = 'data/chromedriver.exe'

# 19세 이상 웹툰 데이터 수집을 위한 selenium 사용
driver = webdriver.Chrome(executable_path=driver_path)

In [3]:
driver.get(url_main + 'webtoon/finish')

직접 로그인 진행

# 웹툰 상세 페이지 이동

In [91]:
r = requests.get(url_main + 'webtoon/finish').text
soup = BeautifulSoup(r, 'lxml')

In [92]:
a = soup.select('div.thumb a')[0]
a

<a href="/webtoon/list?titleId=780984" onclick="nclk_v2(event,'lst.img','','1')" title="엑스트라 데이즈">
<img alt="엑스트라 데이즈" height="90" onerror="this.src='https://ssl.pstatic.net/static/comic/images/migration/common/blank.gif'" src="https://shared-comic.pstatic.net/thumb/webtoon/780984/thumbnail/thumbnail_IMAG10_59b0aaa5-0153-4e9f-b34f-1c860b46d347.jpg" title="엑스트라 데이즈" width="83"/>
<span class="mask"></span>
</a>

In [93]:
a.attrs['href']

'/webtoon/list?titleId=780984'

In [98]:
stars = soup.select('div.rating_type strong')
stars[:10]

[<strong>9.65</strong>,
 <strong>9.92</strong>,
 <strong>9.20</strong>,
 <strong>9.91</strong>,
 <strong>9.87</strong>,
 <strong>9.94</strong>,
 <strong>9.97</strong>,
 <strong>9.92</strong>,
 <strong>9.73</strong>,
 <strong>9.97</strong>]

In [100]:
stars = [float(star.get_text()) for star in stars]
stars[:10]

[9.65, 9.92, 9.2, 9.91, 9.87, 9.94, 9.97, 9.92, 9.73, 9.97]

In [56]:
def get_webtoon_titleId(url_main=url_main):
    r = requests.get(url_main + 'webtoon/finish').text
    soup = BeautifulSoup(r, 'lxml')
    a_list = soup.select('div.thumb a')
    epi_url_list = []
    comment_url_list = []
    
    for a in a_list:
        epi_url = url_main + 'webtoon/detail' + a.attrs['href'].split('list')[-1]
        comment_url = url_main + 'comment/comment' + a.attrs['href'].split('list')[-1]
        epi_url_list.append(epi_url)
        comment_url_list.append(comment_url)
        
    return epi_url_list, comment_url_list

In [57]:
epi_url_list, comment_url_list = get_webtoon_titleId()

In [58]:
epi_url_list[:10]

['https://comic.naver.com/webtoon/detail?titleId=780984',
 'https://comic.naver.com/webtoon/detail?titleId=733079',
 'https://comic.naver.com/webtoon/detail?titleId=780857',
 'https://comic.naver.com/webtoon/detail?titleId=780983',
 'https://comic.naver.com/webtoon/detail?titleId=776541',
 'https://comic.naver.com/webtoon/detail?titleId=755674',
 'https://comic.naver.com/webtoon/detail?titleId=655749',
 'https://comic.naver.com/webtoon/detail?titleId=751625',
 'https://comic.naver.com/webtoon/detail?titleId=778741',
 'https://comic.naver.com/webtoon/detail?titleId=728015']

In [59]:
comment_url_list[:10]

['https://comic.naver.com/comment/comment?titleId=780984',
 'https://comic.naver.com/comment/comment?titleId=733079',
 'https://comic.naver.com/comment/comment?titleId=780857',
 'https://comic.naver.com/comment/comment?titleId=780983',
 'https://comic.naver.com/comment/comment?titleId=776541',
 'https://comic.naver.com/comment/comment?titleId=755674',
 'https://comic.naver.com/comment/comment?titleId=655749',
 'https://comic.naver.com/comment/comment?titleId=751625',
 'https://comic.naver.com/comment/comment?titleId=778741',
 'https://comic.naver.com/comment/comment?titleId=728015']

In [61]:
driver.get(epi_url_list[0] + '&no=1')

# 회차별 댓글 수집
- 2013년 5월 이후 업로드된 작품만 수집(베댓 시스템 생긴 시점)

## 제목, 날짜 수집

In [62]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup.select('.date')

[<dd class="date">2021.10.11</dd>]

In [63]:
soup.select('.date')[0].get_text()

'2021.10.11'

In [64]:
import datetime

date = soup.select('.date')[0].get_text()
dt_test = datetime.date(int(date.split('.')[0]), int(date.split('.')[1]), int(date.split('.')[2]))
dt_test

datetime.date(2021, 10, 11)

In [65]:
dt_test > datetime.date(2013, 5, 1)

True

In [69]:
soup.select('.detail')[0].get_text().split('\t')[0].strip()

'엑스트라 데이즈'

## 댓글 수집

In [70]:
driver.get(comment_url_list[0] + '&no=1')

In [72]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup.select('.u_cbox_contents')

[<span class="u_cbox_contents" data-lang="ko" style="">아 근데 실제로 저런 기술이 있다면 쫌 싸할것같다.. 내가 죽어도 사람들은 나랑 똑같이 생긴 로봇이나 주문할 생각 하고 있을거 아니야..</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">헉 완전 명작 느낌</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">아니 이게 네이버로... 댑악적ㅠㅠ</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">아니영 작가님 작품이라니... 고민없이 쿠키갑니다.. 저는 당신의 제빵사</span>]

In [74]:
soup.select('.u_cbox_cnt_recomm') # 베댓 추천 수

[<em class="u_cbox_cnt_recomm">57</em>,
 <em class="u_cbox_cnt_recomm">39</em>,
 <em class="u_cbox_cnt_recomm">8</em>,
 <em class="u_cbox_cnt_recomm">8</em>]

In [75]:
soup.select('.u_cbox_cnt_unrecomm') # 베댓 비추 수

[<em class="u_cbox_cnt_unrecomm">0</em>,
 <em class="u_cbox_cnt_unrecomm">0</em>,
 <em class="u_cbox_cnt_unrecomm">0</em>,
 <em class="u_cbox_cnt_unrecomm">0</em>]

In [80]:
# 베스트 댓글에서 전체 댓글로 변경
driver.find_element_by_css_selector('#cbox_module_wai_u_cbox_sort_option_tab2 > span.u_cbox_sort_label').click()

In [81]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup.select('.u_cbox_contents')

[<span class="u_cbox_contents" data-lang="ko" style="">이 웹툰은 개재밌을거라고 나한테 텔레파시가왔다</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">이게..... 이게 네이버에 오다니!!!!! 너무 감사합니다 진짜ㅠㅠ 케이툰에서 정말 재밌게 봤었어요ㅠㅠ</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">한 번쯤 생각해본 적 있는 상황인데..만약에 실제로 저런 기술이 도입된다면 어떨까?</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">대박재밌어요!</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">있으면 진짜 세대갈등 뺨치게 갈등 커질듯</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">그림체가 눈에 익는다고 생각했는데 라일락200%작가님이셨군요!!!</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">소재벌써좋다</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">왜 하루만에 모든화가 올라온거임? 나만그렇게 보이나</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">썸네일 윙크가 너무 앙큼해서 보러옴ㅎ</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">죽은 사람은 보내줘야지...ㅠㅠ 오히려 붙잡고있는게 정신 건강에 해로울텐데</span>,
 <span class="u_cbox_contents" data-lang="ko" style="">영광의교실이랑 오렌지 그 뭐냐

In [82]:
soup.select('.u_cbox_count') # 전체 댓글 수

[<span class="u_cbox_count">41</span>]

In [87]:
driver.get(epi_url_list[0] + '&no=1')

soup = BeautifulSoup(driver.page_source, 'html.parser')
soup.select('.pointTotalPerson em') # 위 아래에 있어서 2개 출력 (평가한 인원 수)

[<em>101</em>, <em>101</em>]

In [90]:
soup.select('#topPointTotalNumber')[0].get_text() # 평점

'9.62'

## 함수 생성

In [123]:
import time

basedate = datetime.date(2013, 5, 1)

def get_comment_epi(epi_url, comment_url, star, after=basedate, no=1):
    """
    epi_url : 회차 base url (작품 제목과 날짜를 가져오기 위한 url)
    comment_url : 회차 댓글 url (베스트 댓글 수 등을 가져오기 위한 url)
    star : 평균 평점 (예측을 위한 값(y))
    after : 베댓 시스템이 생긴 2013년 5월 이후로만 수집
    no : 회차
    """
    
    # 우선 날짜를 추출 하여 2013년 5월 전에 올라온 회차면 pass
    driver.get(epi_url + '&no={}'.format(no))
    time.sleep(0.5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    date = soup.select('.date')[0].get_text()
    date = datetime.date(int(date.split('.')[0]), int(date.split('.')[1]), int(date.split('.')[2]))
    
    if date < basedate:
        return
    
    # 제목과 평점, 평가한 인원 수 가져오기
    title = soup.select('.detail')[0].get_text().split('\t')[0].strip()
    score = float(soup.select('#topPointTotalNumber')[0].get_text())
    score_num = int(soup.select('.pointTotalPerson em')[0].get_text())
    
    # 베스트 댓글 수, 내용, 추천 합, 비추천 합 추출
    driver.get(comment_url + '&no={}'.format(no))
    time.sleep(0.5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    best_comments = soup.select('.u_cbox_contents')
    
    best_comments_num = len(best_comments)
    best_comments = [comment.get_text() for comment in best_comments]
    best_comments = '\n'.join(best_comments) # 댓글을 하나로 합침
    
    recommen = soup.select('.u_cbox_cnt_recomm')
    recommen = [int(num.get_text()) for num in recommen]
    sum_recommen = sum(recommen)
    
    unrecomm = soup.select('.u_cbox_cnt_unrecomm')
    unrecomm = [int(num.get_text()) for num in unrecomm]
    sum_unrecomm = sum(unrecomm)
    
    # 전체 댓글 수, 최근 15개 댓글 내용 추출
    comments_num = int(soup.select('.u_cbox_count')[0].get_text().replace(',', ''))
    
    driver.find_element_by_css_selector('#cbox_module_wai_u_cbox_sort_option_tab2 > span.u_cbox_sort_label').click()
    time.sleep(0.5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    comments_ = soup.select('.u_cbox_contents')
    comments_ = [comment.get_text() for comment in comments_]
    comments_ = '\n'.join(comments_)
    
    return [title, score, score_num, best_comments_num, best_comments, sum_recommen, sum_unrecomm, comments_num, comments_, star]

In [124]:
# 테스트
result = get_comment_epi(epi_url_list[0], comment_url_list[0], stars[0])
result

['엑스트라 데이즈',
 9.62,
 101,
 6,
 '아 근데 실제로 저런 기술이 있다면 쫌 싸할것같다.. 내가 죽어도 사람들은 나랑 똑같이 생긴 로봇이나 주문할 생각 하고 있을거 아니야..\n헉 완전 명작 느낌\n아니영 작가님 작품이라니... 고민없이 쿠키갑니다.. 저는 당신의 제빵사\n아니 이게 네이버로... 댑악적ㅠㅠ\n그림체가 눈에 익는다고 생각했는데 라일락200%작가님이셨군요!!!\n영광의교실이랑 오렌지 그 뭐냐 암튼 그작가님!',
 126,
 0,
 42,
 '버키 왜 하나만 블러예욬ㅋㅋㅋㅋㅋ\n이 웹툰은 개재밌을거라고 나한테 텔레파시가왔다\n이게..... 이게 네이버에 오다니!!!!! 너무 감사합니다 진짜ㅠㅠ 케이툰에서 정말 재밌게 봤었어요ㅠㅠ\n한 번쯤 생각해본 적 있는 상황인데..만약에 실제로 저런 기술이 도입된다면 어떨까?\n대박재밌어요!\n있으면 진짜 세대갈등 뺨치게 갈등 커질듯\n그림체가 눈에 익는다고 생각했는데 라일락200%작가님이셨군요!!!\n소재벌써좋다\n왜 하루만에 모든화가 올라온거임? 나만그렇게 보이나\n썸네일 윙크가 너무 앙큼해서 보러옴ㅎ\n죽은 사람은 보내줘야지...ㅠㅠ 오히려 붙잡고있는게 정신 건강에 해로울텐데\n영광의교실이랑 오렌지 그 뭐냐 암튼 그작가님!\n라일락 200%,영광의 교실 작가님 ㅠㅠ\n아하 제목만보고 저 소다가 사람이된거같은 상큼한 애가가정폭력을 하는 부모를 갱생시키는 그런류인가 생각했는데이런 느낌의 작품이라니 미쳤다 미쳤어!!!\n와 소재봐,,',
 9.65]

In [125]:
# 테스트2 (2012년 작품)
result = get_comment_epi('https://comic.naver.com/webtoon/detail?titleId=318995', 'https://comic.naver.com/comment/comment?titleId=318995', 0)
result

In [126]:
type(result)

NoneType

In [131]:
# 코드 돌리기 전에 반드시 클린봇 OFF로!!

import pandas as pd
from tqdm.auto import tqdm

results = []
for epi_url, comment_url, star in tqdm(zip(epi_url_list, comment_url_list, stars), total=len(epi_url_list)):
    result = get_comment_epi(epi_url, comment_url, star)
    if result is None:
        continue
    results.append(result)
    
df = pd.DataFrame(results, columns=['title', 'score', 'score_num', 'best_num', 'best', 'best_recomm', 'best_unrecomm', 'comment_num', 'comment', 'star'])
df.head()

  0%|          | 0/1061 [00:00<?, ?it/s]

,title,score,score_num,best_num,best,best_recomm,best_unrecomm,comment_num,comment,star
0,엑스트라 데이즈,9.62,101,6,아 근데 실제로 저런 기술이 있다면 쫌 싸할것같다.. 내가 죽어도 사람들은 나랑 똑...,126,0,42,버키 왜 하나만 블러예욬ㅋㅋㅋㅋㅋ\n이 웹툰은 개재밌을거라고 나한테 텔레파시가왔다\...,9.65
1,원수를 사랑하라,9.95,31400,15,근데 여주 대단하다...엄마랑 아빠가 저런 사람들인데 멀끔히 차려입고 면접보는 거면...,129323,3789,984,그림체 마음에 드네 \n그림체 미친\n면접관중에 존나 보기싫은얼굴이보이는데?\n누가...,9.92
2,우투리,8.50,159,4,"학원액션물 만화계의 거장 임재원님의 2번째 작품 우투리입니다.... 스토리, 그림체...",194,5,51,진짜 퀄리티 말도 안되는데 덴마 이후로 미완결 작품은 그냥 혐오스럽다\n우와아아아!...,9.20
3,롭플롭,9.78,290,5,"나 이 작가 작품 좋아하네.... 잭슨의 관, 데빌샷 보고 바로 들어옴 \n잭슨의 ...",410,2,64,작가님 어케 이렇게 그천이신거에요\n노랑머리 친구 명암때문인진 몰라도장례식 이후에 ...,9.91
4,약초마을 연쇄살초사건,9.73,10226,15,아니ㅋㅋㅋㅋㅋㅋ작가님ㅋㅋㅋㅋㅋㅋ이제 작물 쪽으로 길을 트신건가요ㅋㅋㅋㅋㅋㅋㅋㅋ\n팀...,62851,254,1463,ㅋㅋㅋㅋㅋㅋㅋㅋ\n조낸 무서운 이야기인데 약초란 사실과 그림체 때문에 잔혹함이 전달...,9.87


In [134]:
df.to_csv('data/naver_comment.csv', encoding='utf8', index=False)

In [136]:
df.to_csv('data/naver_comment3.csv', encoding='utf-8-sig', index=False)

In [ ]:
driver.close()